In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = mls
band = [8]
commitnumber = de5d43e
molecule = {'h2o': 'atmpro'}
tsfc = 294


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band6
  atmpro = mls
  band = 6
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 5e-25
  molecule = h2o
  ng_adju = [0]
  ng_refs = [4]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(600, 250)]
  tsfc = 294
  vmax = 1215
  vmin = 1100
  w_diffuse = [(1.66, 1.66, 1.7, 1.8)]
  wgt = [(0.3, 0.45, 0.6, 0.95)]


# Cooling Rate by Layer

CRD               CLIRAD `coolr_bands`              
            layer        coolrg                layer        coolrg
pressure                                                          
0.000312      1.0 -2.325424e-05                  NaN           NaN
0.000312      NaN           NaN                  1.0 -2.150232e-06
0.000750      2.0 -2.305708e-05                  2.0  1.978892e-05
0.001052      3.0 -2.261218e-05                  3.0 -1.040337e-05
0.001476      4.0 -2.211744e-05                  4.0 -1.321134e-05
0.002070      5.0 -2.157677e-05                  5.0 -1.341646e-05
0.002904      6.0 -2.099863e-05                  6.0 -1.331407e-05
0.004074      7.0 -2.038466e-05                  7.0 -1.348321e-05
0.005714      8.0 -1.973184e-05                  8.0 -1.327403e-05
0.008015      9.0 -1.903195e-05                  9.0 -1.336512e-05
0.011243     10.0 -1.829874e-05                 10.0 -1.316047e-05
0.015771     11.0 -1.752827e-05                 11.0 -1.316558e-05
0.022122     12.0 -1.670660e-05                 12.0 -1.300590e-05
0.031031     13.0 -1.584098e-05                 13.0 -1.277311e-05
0.043528     14.0 -1.486255e-05                 14.0 -9.744282e-06
0.061057     15.0 -1.164532e-05                 15.0 -5.982607e-06
0.085645     16.0 -3.505993e-07                 16.0 -5.558291e-06
0.120136     17.0  2.437946e-05                 17.0 -2.773504e-06
0.168516     18.0  6.418822e-05                 18.0  6.047584e-06
0.236378     19.0  1.248647e-04                 19.0  2.037576e-05
0.331549     20.0  2.141946e-04                 20.0  4.205463e-05
0.465100     21.0  3.421664e-04                 21.0  7.381060e-05
0.652400     22.0  5.202400e-04                 22.0  1.191791e-04
0.915100     23.0  7.053925e-04                 23.0  1.665652e-04
1.283650     24.0  7.200040e-04                 24.0  1.720031e-04
1.800600     25.0  5.677113e-04                 25.0  1.349263e-04
2.525700     26.0  4.088647e-04                 26.0  9.495780e-05
3.542800     27.0  2.819393e-04                 27.0  6.267148e-05
4.969550     28.0  1.874858e-04                 28.0  3.848946e-05
6.970850     29.0  1.210936e-04                 29.0  2.128096e-05
9.778100     30.0  7.450739e-05                 30.0  8.967640e-06
13.715850    31.0  4.040506e-05                 31.0 -3.369951e-07
19.239350    32.0  1.602507e-05                 32.0 -7.330805e-06
26.987250    33.0  1.007268e-06                 33.0 -1.205658e-05
37.855300    34.0 -7.935108e-06                 34.0 -1.537630e-05
53.100050    35.0 -1.488345e-05                 35.0 -1.843570e-05
73.887500    36.0 -2.051687e-05                 36.0 -2.140938e-05
97.662500    37.0 -2.377461e-05                 37.0 -2.369576e-05
121.437500   38.0 -2.548166e-05                 38.0 -2.529278e-05
145.212500   39.0 -2.708272e-05                 39.0 -2.661121e-05
168.987500   40.0 -2.890967e-05                 40.0 -2.876743e-05
192.762500   41.0 -4.001848e-05                 41.0 -4.332857e-05
216.537500   42.0 -7.557149e-05                 42.0 -1.042860e-04
240.312500   43.0 -8.088149e-05                 43.0 -1.912507e-04
264.087500   44.0  1.834700e-05                 44.0 -2.622154e-04
287.862500   45.0  3.010791e-04                 45.0 -2.215918e-04
311.637500   46.0  7.257637e-04                 46.0  7.075631e-05
335.412500   47.0  1.296967e-03                 47.0  5.989796e-04
359.187500   48.0  2.078026e-03                 48.0  1.392958e-03
382.962500   49.0  3.100541e-03                 49.0  2.471788e-03
406.737500   50.0  4.391978e-03                 50.0  3.806227e-03
430.512500   51.0  5.969354e-03                 51.0  5.332548e-03
454.287500   52.0  7.709830e-03                 52.0  6.881099e-03
478.062500   53.0  9.690026e-03                 53.0  8.491892e-03
501.837500   54.0  1.200194e-02                 54.0  1.027262e-02
525.612500   55.0  1.467523e-02                 55.0  1.231626e-02
549.387500   56.0  1.774709e-02                 56.0  1.48322

# Fluxes by Layer

CRD                              CLIRAD  \
                         flug          fldg      fnetg       flug   
pressure     level                                                  
0.000000e+00 1     -21.780996  0.000000e+00 -21.780996        NaN   
1.000000e-08 1            NaN           NaN        NaN -21.880928   
6.244000e-04 2     -21.780996  1.378786e-09 -21.780996 -21.880928   
8.759000e-04 3     -21.780996  1.942834e-09 -21.780996 -21.880928   
1.228600e-03 4     -21.780996  2.760500e-09 -21.780996 -21.880928   
1.723400e-03 5     -21.780996  3.948470e-09 -21.780996 -21.880928   
2.417400e-03 6     -21.780996  5.673999e-09 -21.780996 -21.880928   
3.390900e-03 7     -21.780996  8.180604e-09 -21.780996 -21.880928   
4.756500e-03 8     -21.780996  1.182119e-08 -21.780996 -21.880928   
6.672000e-03 9     -21.780996  1.710783e-08 -21.780996 -21.880928   
9.358900e-03 10    -21.780996  2.478416e-08 -21.780996 -21.880928   
1.312780e-02 11    -21.780996  3.592900e-08 -21.780996 -21.880928   
1.841450e-02 12    -21.780996  5.210796e-08 -21.780996 -21.880928   
2.583020e-02 13    -21.780996  7.559244e-08 -21.780996 -21.880928   
3.623230e-02 14    -21.780996  1.096784e-07 -21.780996 -21.880928   
5.082340e-02 15    -21.780996  1.592564e-07 -21.780996 -21.880928   
7.129060e-02 16    -21.780996  2.365699e-07 -21.780996 -21.880928   
1.000000e-01 17    -21.780997  3.798058e-07 -21.780996 -21.880928   
1.402710e-01 18    -21.780997  6.741545e-07 -21.780996 -21.880928   
1.967600e-01 19    -21.780997  1.276706e-06 -21.780996 -21.880928   
2.759970e-01 20    -21.780997  2.496307e-06 -21.780994 -21.880928   
3.871000e-01 21    -21.780997  4.935495e-06 -21.780992 -21.880928   
5.431000e-01 22    -21.780995  9.761896e-06 -21.780985 -21.880928   
7.617000e-01 23    -21.780991  1.915959e-05 -21.780972 -21.880927   
1.068500e+00 24    -21.780982  3.606646e-05 -21.780946 -21.880926   
1.498800e+00 25    -21.780970  6.004530e-05 -21.780910 -21.880924   
2.102400e+00 26    -21.780956  8.737486e-05 -21.780869 -21.880922   
2.949000e+00 27    -21.780944  1.164266e-04 -21.780828 -21.880922   
4.136600e+00 28    -21.780935  1.466151e-04 -21.780788 -21.880922   
5.802500e+00 29    -21.780929  1.777180e-04 -21.780751 -21.880925   
8.139200e+00 30    -21.780928  2.100141e-04 -21.780718 -21.880930   
1.141700e+01 31    -21.780933  2.439066e-04 -21.780689 -21.880939   
1.601470e+01 32    -21.780946  2.794329e-04 -21.780667 -21.880954   
2.246400e+01 33    -21.780971  3.168527e-04 -21.780655 -21.880975   
3.151050e+01 34    -21.781012  3.584014e-04 -21.780654 -21.881006   
4.420010e+01 35    -21.781073  4.077982e-04 -21.780665 -21.881052   
6.200000e+01 36    -21.781165  4.676821e-04 -21.780697 -21.881121   
8.577500e+01 37    -21.781293  5.380307e-04 -21.780755 -21.881218   
1.095500e+02 38    -21.781426  6.049111e-04 -21.780822 -21.881321   
1.333250e+02 39    -21.781566  6.725131e-04 -21.780893 -21.881431   
1.571000e+02 40    -21.781710  7.408926e-04 -21.780970 -21.881545   
1.808750e+02 41    -21.781867  8.158474e-04 -21.781051 -21.881671   
2.046500e+02 42    -21.782118  9.538267e-04 -21.781164 -21.881874   
2.284250e+02 43    -21.782810  1.433532e-03 -21.781377 -21.882455   
2.522000e+02 44    -21.784411  2.806818e-03 -21.781604 -21.883834   
2.759750e+02 45    -21.787632  6.079432e-03 -21.781553 -21.886673   
2.997500e+02 46    -21.793450  1.274490e-02 -21.780705 -21.891884   
3.235250e+02 47    -21.801633  2.297224e-02 -21.778661 -21.899280   
3.473000e+02 48    -21.812160  3.715205e-02 -21.775008 -21.908838   
3.710750e+02 49    -21.825364  5.620823e-02 -21.769156 -21.920879   
3.948500e+02 50    -21.841569  8.114508e-02 -21.760424 -21.935725   
4.186250e+02 51    -21.861077  1.130225e-01 -21.748054 -21.953658   
4.424000e+02 52    -21.884187  1.529443e-01 -21.731243 -21.974946   
4.661750e+02 53    -21.910642  2.011129e-01 -21.709529 -21.999364   
4.899500e+02 54    -21.940377  2.581373e-01 -21.682239 -22.026865   
5.137250e+02 55    -21.9

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')